In [1]:
import zipfile
import os
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import optimizers
from keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten, concatenate
from tensorflow.keras import regularizers
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.utils.vis_utils import plot_model
import pylab as pl
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [2]:
path = '../input/petfinder-pawpularity-score/'
training_dir = path + '/train'
test_dir = path + '/test'

df_train = pd.read_csv(path + '/train.csv')
df_test = pd.read_csv(path + '/test.csv')
df_train.head()

In [3]:
# Load the images and the labels
train_imgs = []
for img in sorted(os.listdir(path + '/train')):
    pic = image.load_img((path + '/train/' + img), target_size=(300, 300))
    train_imgs.append(np.array(pic))

In [4]:
# Load test images
test_imgs = []
for img in sorted(os.listdir(path + '/test')):
    pic = image.load_img((path + '/test/' + img), target_size=(300, 300))
    test_imgs.append(np.array(pic))

In [5]:
# Train labels
# y_train = np.array(df_train['Pawpularity'])/100 prova senza divisione
Y = np.array(df_train['Pawpularity'])

df_train.drop('Pawpularity', axis=1, inplace=True)
df_train.drop('Id', axis=1, inplace=True)

# Images Train and validation split 
x_train, x_val, y_train, y_val = train_test_split(train_imgs, Y, test_size=0.1, random_state=11)

# Dataframe train and validation split
df_x_train, df_x_val, df_y_train, df_y_val = train_test_split(df_train, Y, test_size=0.1, random_state=11)

# CNN

In [6]:
# Load a pre-trained neural network to use for transfer learning
base_net = keras.applications.EfficientNetB3(
    input_shape=(300,300,3),
    weights='../input/noisystudentkeras/noisystudent_efficientnetb3_notop.h5',
    include_top=False,
    drop_connect_rate=0.4,
    pooling='avg'
)

# Set the layers of the base net to not be trained
for layer in base_net.layers:
  layer.trainable=False
    


In [7]:
x = Flatten()(base_net.layers[-1].output)
x = keras.layers.Dense(1024, activation='relu')(x)
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Dense(512, activation='relu')(x)
x = keras.layers.Dropout(0.3)(x)
x = keras.layers.Dense(128, activation='relu')(x)
x = keras.layers.Dropout(0.1)(x)
x = keras.layers.Dense(1, activation='linear')(x)
outputs = x

OPT = tf.keras.optimizers.Adam()

cnn = keras.Model(inputs=base_net.inputs, outputs=outputs)
cnn.compile(loss='mean_squared_error', optimizer=OPT, metrics=['RootMeanSquaredError'])
#cnn.summary()

In [8]:
hist_cnn = cnn.fit(np.array(x_train), y_train, epochs=15, validation_data=(np.array(x_val), y_val), callbacks=[EarlyStopping('val_loss', mode='auto', patience=5, restore_best_weights=True, verbose=1)])

In [9]:

def plot_hist(hist):
    plt.plot(hist.history["root_mean_squared_error"])
    plt.plot(hist.history["val_root_mean_squared_error"])
    plt.title("model rmse")
    plt.ylabel("rmse")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()
    
    # summarize history for loss
    plt.plot(hist.history['loss'])
    plt.plot(hist.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()


plot_hist(hist_cnn)

# MLP

In [10]:
dims = df_x_train.shape[1]

mlp = Sequential()
mlp.add(Dense(32, activation='relu', input_shape=(dims,)))
mlp.add(Dropout(0.2))
mlp.add(Dense(8, activation='relu'))
mlp.add(Dropout(0.1))
mlp.add(Dense(1, activation="linear"))


# compile: optimizer & losses/metrics
mlp.compile(loss='mean_squared_error', optimizer=OPT, metrics=['RootMeanSquaredError'])
mlp.summary()

In [11]:
hist_FC = mlp.fit(df_x_train, df_y_train, epochs=15, batch_size=32, validation_data=(df_x_val, df_y_val), callbacks=[EarlyStopping('val_loss', mode='auto', patience=3, restore_best_weights=True, verbose=1)])

In [12]:
plot_hist(hist_FC)

# CNN + MLP

In [13]:
combined = concatenate([mlp.output, cnn.output])

# apply a FC layer and then a regression prediction on the
# combined outputs
z = Dense(2, activation="relu")(combined)
z = Dense(1, activation="linear")(z)
model = Model(inputs=[mlp.input, cnn.input], outputs=z)
model.compile(loss='mean_squared_error', optimizer=OPT, metrics=['RootMeanSquaredError'])
#plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [14]:

earlyStopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
#callbacks_list = [checkpoint, earlyStopping]

# train the model
hist = model.fit(x=[df_x_train, np.array(x_train)], 
          y=y_train, 
          epochs=15, 
          batch_size=32, 
          validation_data=([df_x_val, np.array(x_val)], y_val),  
          callbacks=[earlyStopping])


In [15]:
plot_hist(hist)

In [16]:
df_x_test = df_test.loc[:,'Subject Focus':'Blur']
pred = model.predict([df_x_test, np.array(test_imgs)])
submission = pd.DataFrame(df_test['Id'])
submission['Pawpularity'] = pred
submission.to_csv('submission.csv', index=False)

In [17]:
submission.head(5)

# Models performance

In [29]:
loss_cnn = min(hist_cnn.history['val_loss'])
rmse_cnn = min(hist_cnn.history['val_root_mean_squared_error'])
print("validation loss CNN model:", loss_cnn)
print("validation rmse CNN model:", rmse_cnn)

print('\n-----------------------------------\n')

loss_mlp = min(hist_FC.history['val_loss'])
rmse_mlp = min(hist_FC.history['val_root_mean_squared_error'])
print("validation loss MLP model:", loss_mlp)
print("validation rmse MLP model:", rmse_mlp)

print('\n-----------------------------------\n')

loss_final = min(hist.history['val_loss'])
rmse_final = min(hist.history['val_root_mean_squared_error'])
print("validation loss final model:", loss_final)
print("validation rmse final model:", rmse_final)

In [19]:
plt.plot(hist.history["root_mean_squared_error"])
plt.plot(hist.history["val_root_mean_squared_error"])
plt.plot(hist_cnn.history["root_mean_squared_error"])
plt.plot(hist_cnn.history["val_root_mean_squared_error"])
plt.plot(hist_FC.history["root_mean_squared_error"])
plt.plot(hist_FC.history["val_root_mean_squared_error"])
plt.title("model rmse")
plt.ylabel("rmse")
plt.xlabel("epoch")
plt.legend(["train_final_model", "validation_final_model", "train_cnn", "validation_cnn","train_fc", "validation_fc"], loc="upper left")
plt.show()

# summarize history for loss
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.plot(hist_cnn.history['loss'])
plt.plot(hist_cnn.history['val_loss'])
plt.plot(hist_FC.history['loss'])
plt.plot(hist_FC.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(["train_final_model", "validation_final_model", "train_cnn", "validation_cnn","train_fc", "validation_fc"], loc='upper left')
plt.show()

# Grad CAM

In [20]:
import cv2
import numpy as np
from PIL import Image
import tensorflow as tf
import tensorflow.keras as K
import matplotlib.pyplot as plt
from skimage.transform import resize
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array


def VizGradCAM(model, image, interpolant=0.5, plot_results=True):
    """VizGradCAM - Displays GradCAM based on Keras / TensorFlow models
    using the gradients from the last convolutional layer. This function
    should work with all Keras Application listed here:
    https://keras.io/api/applications/
    Parameters:
    model (keras.model): Compiled Model with Weights Loaded
    image: Image to Perform Inference On
    plot_results (boolean): True - Function Plots using PLT
                            False - Returns Heatmap Array
    Returns:
    Heatmap Array?
    """
    # Sanity Check
    assert (
        interpolant > 0 and interpolant < 1
    ), "Heatmap Interpolation Must Be Between 0 - 1"

    last_conv_layer = next(
        x for x in model.layers[::-1] if isinstance(x, K.layers.Conv2D)
    )
    target_layer = model.get_layer(last_conv_layer.name)

    original_img = image
    img = np.expand_dims(original_img, axis=0)
    prediction = model.predict(img)

    # Obtain Prediction Index
    prediction_idx = np.argmax(prediction)

    # Compute Gradient of Top Predicted Class
    with tf.GradientTape() as tape:
        gradient_model = Model([model.inputs], [target_layer.output, model.output])
        conv2d_out, prediction = gradient_model(img)
        # Obtain the Prediction Loss
        loss = prediction[:, prediction_idx]

    # Gradient() computes the gradient using operations recorded
    # in context of this tape
    gradients = tape.gradient(loss, conv2d_out)

    # Obtain the Output from Shape [1 x H x W x CHANNEL] -> [H x W x CHANNEL]
    output = conv2d_out[0]

    # Obtain Depthwise Mean
    weights = tf.reduce_mean(gradients[0], axis=(0, 1))

    # Create a 7x7 Map for Aggregation
    activation_map = np.zeros(output.shape[0:2], dtype=np.float32)

    # Multiply Weights with Every Layer
    for idx, weight in enumerate(weights):
        activation_map += weight * output[:, :, idx]

    # Resize to Size of Image
    activation_map = cv2.resize(
        activation_map.numpy(), (original_img.shape[1], original_img.shape[0])
    )

    # Ensure No Negative Numbers
    activation_map = np.maximum(activation_map, 0)

    # Convert Class Activation Map to 0 - 255
    activation_map = (activation_map - activation_map.min()) / (
        activation_map.max() - activation_map.min()
    )
    activation_map = np.uint8(255 * activation_map)

    # Convert to Heatmap
    heatmap = cv2.applyColorMap(activation_map, cv2.COLORMAP_JET)

    # Superimpose Heatmap on Image Data
    original_img = np.uint8(
        (original_img - original_img.min())
        / (original_img.max() - original_img.min())
        * 255
    )

    cvt_heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)

    # Enlarge Plot
    plt.rcParams["figure.dpi"] = 100

    if plot_results == True:
        plt.imshow(
            np.uint8(original_img * interpolant + cvt_heatmap * (1 - interpolant))
        )
    else:
        return cvt_heatmap
    


In [38]:
test_img1 = img_to_array(load_img("../input/petfinder-pawpularity-score/train/0075ec6503412f21cf65ac5f43d80440.jpg" , target_size=(300,300)))
test_img2 = img_to_array(load_img("../input/petfinder-pawpularity-score/train/0085bfb9a7ebfd776cb804d8b456bb05.jpg" , target_size=(300,300)))

VizGradCAM(cnn, test_img1)

In [35]:
VizGradCAM(cnn, test_img2)